In [12]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types.{DoubleType, DateType}

import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.ml.feature.MinMaxScaler
import org.apache.spark.sql.functions.udf
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types.{DoubleType, DateType}


In [2]:
val weather_org_df = spark.
        read.
        option("inferSchema", "true").
        option("header","true").
        option("mode","DROPMALFORMED").
        option("delimiter", ",").
        csv("wrangle_final_for_yury.csv")

weather_org_df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


In [3]:
weather_org_df.registerTempTable("Aus_Rain_Data")

In [4]:
val df0 = spark.sql("select * from Aus_Rain_Data")

df0: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


In [5]:
val df1 = df0.withColumn("RainToday_Sc", when(lower(df0("RainToday"))  === "yes", 1).otherwise(0))
/*    .drop("RainToday")
    .withColumnRenamed("RainToday_Sc", "RainToday") */
val df2 = df1.withColumn("RainTomorrow_Sc", when(lower(df1("RainTomorrow"))  === "yes", 1).otherwise(0))
/*    .drop("RainTomorrow")
    .withColumnRenamed("RainTomorrow_Sc", "RainTomorrow") */

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 24 more fields]
df2: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 25 more fields]


In [6]:
val df3 = df2.withColumn("Pressure9am_Sc", df2("Pressure9am").cast(DoubleType))
    .drop("Pressure9am")
    .withColumnRenamed("Pressure9am_Sc", "Pressure9am")

val df4 = df3.withColumn("Pressure3pm_Sc", df3("Pressure3pm").cast(DoubleType))
    .drop("Pressure3pm")
    .withColumnRenamed("Pressure3pm_Sc", "Pressure3pm") 

df3: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 25 more fields]
df4: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 25 more fields]


In [7]:
val min_P9am = df4.agg(min("Pressure9am") as "min_P9am").select(col("min_P9am")).first.getDouble(0)
val max_P9am = df4.agg(max("Pressure9am") as "max_P9am").select(col("max_P9am")).first.getDouble(0)

2020-06-04 12:57:08,571 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


min_P9am: Double = 980.5
max_P9am: Double = 1041.0


In [8]:
val min_P3pm = df4.agg(min("Pressure3pm") as "min_P3pm").select(col("min_P3pm")).first.getDouble(0)
val max_P3pm = df4.agg(max("Pressure3pm") as "max_P3pm").select(col("max_P3pm")).first.getDouble(0)

min_P3pm: Double = 977.1
max_P3pm: Double = 1039.6


In [9]:
val minPRange = 0    //from the report  /* 0 970.0
val maxPRange = 1   //from the report  /* 1 970.0

minPRange: Int = 0
maxPRange: Int = 1


\begin{equation}
  Rescaled_{9am} = \frac{Pressure_i - Pressure_{min9am}}{Pressure_{max9am} - Pressure_{min9am}} * (maxPressureRange - minPressureRange) + minPressureRange
\end{equation}

In [10]:
val df5 = df4.withColumn("Pressure3pm_Sc", 
    (((df4("Pressure3pm") - min_P3pm) / (max_P3pm -min_P3pm)) * (maxPRange - minPRange)) + minPRange)
/*    .drop("Pressure3pm")
    .withColumnRenamed("Pressure3pmTmp", "Pressure3pm") */
val df6 = df5.withColumn("Pressure9am_Sc", 
    (((df5("Pressure9am") - min_P9am) / (max_P9am -min_P9am)) * (maxPRange - minPRange)) + minPRange)
/*.drop("Pressure9am")
    .withColumnRenamed("Pressure9amTmp", "Pressure9am") */

df5: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 26 more fields]
df6: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 27 more fields]


In [13]:
val df7 = df6.withColumn("date_Sc", df6("date").cast(DateType))

df7: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 28 more fields]


In [14]:
val df8 = df7.withColumn("date-day_Sc", dayofmonth(df7("date_Sc")))
val df9 = df8.withColumn("date-month_Sc", month(df8("date_Sc")))
val df10 = df9.withColumn("date-year_Sc", year(df9("date_Sc")))

df8: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 29 more fields]
df9: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 30 more fields]
df10: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 31 more fields]


In [19]:
df10.coalesce(1)
      .write
      .option("header","true")
      .option("sep",",")
      .mode("overwrite")
      .csv("data-cleanup")

In [20]:
!ls /home/data-cleanup

_SUCCESS  part-00000-33387aa0-1431-4692-bb85-e6b4f5bf702b-c000.csv



In [21]:
!mv /home/data-cleanup/*.csv /home/weatherAUS-clean-FINAL.csv

In [23]:
!wc -l   weatherAUS-clean-FINAL.csv
!wc -l   wrangle_final_for_yury.csv

142194 weatherAUS-clean-FINAL.csv

142194 wrangle_final_for_yury.csv



In [24]:
!rm -r /home/data-cleanup/

In [25]:
!ssh yury@10.0.2.15 "docker cp bd_cont:/home/data-cleanup.ipynb /home/yury/Downloads/FIT5202-Project/big_data_tutorials/data-cleanup.ipynb"

In [27]:
!ssh yury@10.0.2.15 "docker cp bd_cont:/home/weatherAUS-clean-FINAL.csv /home/yury/Downloads/FIT5202-Project/big_data_tutorials/weatherAUS-clean-FINAL.csv"